In [3]:
import pickle

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from time import time
import math
from tqdm import tqdm_notebook

In [1]:
class review:
    def __init__(self, pn, name, age, ftype, sex, point, comment):
        self.pn = pn
        self.name = name
        self.age = age
        self.ftype = ftype
        self.sex = sex
        self.point = point
        self.comment = comment
    
    def __repr__(self):
        return '{},{},{},{},{},{}'.format(self.pn, self.name, self.age, self.ftype, self.point, self.comment)

class ingredient():
    def __init__(self, pn, kname, ename, exp, level, degree):
        self.pn = pn
        self.kname = kname
        self.ename = ename
        self.exp = exp
        self.level = level
        self.degree = degree
        
    def __repr__(self):
        return '{},{},{},{},{},{}'.format(self.pn, self.kname, self.ename, self.exp, self.level, self.degree)

def crawl_ingredient(pn):
    # 화장품성분  
    
    info = driver.find_elements_by_xpath('//*[@id="gp-product-detail"]/div/ul[1]/li[2]/div/section[2]/div/div/button')
    if len(info)>0:
        for _ in range(20):
            try:
                info[0].click()
                break
            except:
                sleep(0.5)
                info = driver.find_elements_by_xpath('//*[@id="gp-product-detail"]/div/ul[1]/li[2]/div/section[2]/div/div/button')
                pass
            
    else:
        return []
     
    st = time()
    while(True):        
        measures = driver.find_elements_by_css_selector('section.side-info.ingredient div.list-ingredient')
        if len(measures)>0:
            measures = measures[0]
            measures = measures.find_elements_by_class_name('list')
            break
        if time() - st > 3:
            measures = []
            break
     
    ings =[]
    for m in measures:
        soup = BeautifulSoup(m.get_attribute('innerHTML'), 'lxml')
        icon = soup.select_one('div.list-item > span')
        icontxt = icon.text
        icon = icon['class'][2]

        names = soup.select('div.list-item div.txt p')
        narr = names[0].text.split('\n')
        n1 = narr[0].strip()
        n2 = narr[1].strip()
        n3 = names[1].text.strip() 
        #print(icontxt, icon)
        ing = ingredient(pn, n1, n2, n3, icontxt, icon)
        ings.append(ing)
    return ings

def crawl_reviews(pn):
    driver.get('https://www.glowpick.com/product/{}'.format(pn))     
    uf = 'https://www.glowpick.com/product/{}?rating={}&order={}'  
    
    
    
    # 리뷰 개수 체크용
    count = '#gp-product-detail > div > ul.section-list-wrap.side-bottom > li.section-list-score.side-right > section > div > div.side-right > ul > li:nth-child({}) > div > p'
    creviews = [] 
    
    # 상품설명
    exp = driver.find_element_by_css_selector('#gp-product-detail > div > ul.section-list-wrap.side-bottom > li.section-list-pdt.side-left > section > div > ul > li.desc > p').text

    # 상품 키워드
    tag = driver.find_element_by_css_selector('#gp-product-detail > div > ul.section-list-wrap.side-bottom > li.section-list-pdt.side-left > section > div > ul > li.desc > ul').text
    
    errors = []
    
    pdic = {'gpa-best': 5, 'gpa-good': 4, 'gpa-soso': 3, 'gpa-bad': 2, 'gpa-worst': 1}

    cnts = driver.find_element_by_xpath('//*[@id="gp-product-detail"]/div/ul[1]/li[2]/div/section[1]/div[1]/div/span[3]').text
    cnts = cnts.replace('(','').replace(')','').replace(',','')
    cnts = int(cnts) 
    if cnts != 0:
        for fn1, cidx in zip(range(5, 0, -1), range(1, 6)):  
            for idx, fn2 in enumerate(['create_date_asc','create_date_desc','like_desc','like_asc']):
                url = uf.format(pn, fn1, fn2)
                driver.get(url)
                #sleep(0.5)

                # 앱으로보기 링크 있을경우 끄기
                banners = driver.find_elements_by_xpath('//*[@id="top-downloadbanner"]/button')
                if len(banners)>0:
                    banners[0].click()
    
    
                cnt = driver.find_element_by_css_selector(count.format(cidx)).text
                cnt = int(cnt) 
                if cnt == 0:
                    break
                loop = min(math.ceil(cnt/20), 5)
                noloop1 = noloop2 = False
                if cnt <= 100: noloop1 = True
                if cnt <= 200: noloop2 = True
                    
                    
                nor = driver.find_elements_by_xpath('//*[@id="gp-product-detail"]/div/ul[2]/li[5]/div/button')
                if len(nor)==0:
                    st = time()
                    # 리뷰 리스트가 없으면 찾을때까지 대기
                    while(True):
                        rlist = driver.find_elements_by_class_name('review-list-wrap')
                        if len(rlist)>0:
                            if (rlist[0].text != ''):
                                body = driver.find_element_by_css_selector('body')
                                body.click()
                                break
                        else:
                            driver.get(url)
                            sleep(0.5)

                        if time() - st > 30:
                            errors.append([pn, fn1, fn2])
                            break
                else:
                    break
                    
  
                # 최대 100개, 스크롤 해야 로드하기 때문에 end키 조작으로 리뷰 불러오기
                st = time()
                while(True):
                    try:
                        body.send_keys(Keys.END) 
                        reviews = driver.find_elements_by_css_selector('ul.review-list-wrap li div.list-item') 

                        if time() - st > 3:
                            body.send_keys(Keys.PAGE_UP)
                            sleep(0.1) 
                            body.send_keys(Keys.END)
                            st = time()

                        if len(reviews) > (loop-1) * 20:
                            break

                    except Exception as e:
                        body = driver.find_element_by_css_selector('body')
                        body.click()


                #print(len(reviews))
                # 리뷰 크롤링
                # beautifulsoup이 좀더 빠르다
                for r in reviews:
                    soup = BeautifulSoup(r.get_attribute('innerHTML'), 'lxml')
                    user = soup.select_one('.user .user-info')
                    name = user.select_one('.name').text
                    agetype = user.select_one('.info .txt')
                    arr = agetype.text.split('·')
                    age = int(arr[0].replace('세',''))
                    ftype = arr[1] 
                    sex = agetype.select_one('span')['class']
                    point = user.select_one('.info .label span')['class'][1]
                    point = pdic[point]

                    comment = soup.select_one('.review').text

                    cr = review(pn, name, age, ftype, sex, point, comment)
                    creviews.append(cr)

                if noloop1: break 
                if noloop2 & (idx >= 1):break

        #rank = driver.find_element_by_css_selector('#gp-product-detail > div > ul.section-list-wrap.side-top > li.section-list-img.side-left > div > section > ul > li > span').text
        #print(rank) 
     
    ing = crawl_ingredient(pn)
    
    return exp, tag, creviews, ing, errors

In [21]:
df = pd.read_csv('data/rank.csv')
df.head()

,cate,cid,rank,change,diff,pn,company,name,volume,price
0,미스트,1,1,NaN,NaN,95080,홀리카홀리카,굳세라 수퍼 세라마이드 미스트,120ml,"15,000원"
1,미스트,1,2,NaN,NaN,27484,한율,어린쑥 수분 진정수,150ml,"25,000원"
2,미스트,1,3,NaN,NaN,51533,CNP(차앤박),프로폴리스 앰플 미스트,100ml,"15,000원"
3,미스트,1,4,NaN,NaN,58165,비어썸(Beausome),오우썸 하이드레이팅 오가닉 더블미스트,120ml,"22,000원"
4,미스트,1,5,NaN,NaN,99864,보타닉힐 보,더마 인텐시브 판테놀 크림 미스트,120ml,"15,000원"


In [4]:
driver = webdriver.Chrome()

In [5]:
reviw_res = []
for idx in tqdm_notebook(df[df['cate']=='스킨'].index):
    pn = df.loc[idx, 'pn']
    res = crawl_reviews(pn)
    reviw_res.append(res)

In [24]:
len(data_skin), len(data_lotion), len(data_cream), len(data_faceoil), len(data_mist)

(200, 200, 200, 100, 100)

In [8]:
with open('data/skin.plk', 'rb') as f:
    data_skin = pickle.load(f)
    
with open('data/lotion.plk', 'rb') as f:
    data_lotion = pickle.load(f)
    
with open('data/mist.plk', 'rb') as f:
    data_mist = pickle.load(f)

with open('data/cream.plk', 'rb') as f:
    data_cream = pickle.load(f)

with open('data/faceoil.plk', 'rb') as f:
    data_faceoil = pickle.load(f)

In [82]:
rres = []
for data in [data_skin, data_lotion, data_mist, data_cream, data_faceoil]:
    for dt in data:
        rs = dt[2]
        for r in rs:
            rres.append([r.pn, r.name, r.age, r.ftype, r.point, r.comment])

In [85]:
total_review = pd.DataFrame(rres, columns=['pn','author','age','ftype','point','ment'])

In [86]:
total_review.to_csv('data/total_review.csv', index=False)

In [95]:
rres = []
for data in [data_skin, data_lotion, data_mist, data_cream, data_faceoil]:
    for dt in data:
        rs = dt[3]
        for r in rs:
            rres.append([r.pn, r.kname, r.ename, r.exp, r.level, r.degree])

In [98]:
total_ing = pd.DataFrame(rres, columns=['pn','kname','ename','exp','level','degree'])
total_ing.to_csv('data/total_ing.csv',index=False)

In [88]:
dfres = []
for cate, data in zip(['스킨','로션','미스트','크림','페이스오일'], [data_skin, data_lotion, data_mist, data_cream, data_faceoil]):
    df_skin1 = df[df['cate']==cate].reset_index(drop=True)
    df_skin2 = pd.DataFrame(data, columns=['exp','tags','r','i','e'])
    df_skinm = pd.concat([df_skin1, df_skin2], axis=1)
    df_skinm = df_skinm.drop(columns=['r','i','e'])
    dfres.append(df_skinm)

In [92]:
total_df = pd.concat(dfres)

In [93]:
total_df.to_csv('data/total_df.csv', index=False)

In [50]:
# class review:
#     def __init__(self, pn, name, age, ftype, sex, point, comment):
#         self.pn = pn
#         self.name = name
#         self.age = age
#         self.ftype = ftype
#         self.sex = sex
#         self.point = point
#         self.comment = comment
    
#     def __repr__(self):
#         return '{},{},{},{},{},{}'.format(self.pn, self.name, self.age, self.ftype, self.point, self.comment)

# class ingredient():
#     def __init__(self, pn, kname, ename, exp, level, degree):
#         self.pn = pn
#         self.kname = kname
#         self.ename = ename
#         self.exp = exp
#         self.level = level
#         self.degree = degree
        
#     def __repr__(self):
#         return '{},{},{},{},{},{}'.format(self.pn, self.kname, self.ename, self.exp, self.level, self.degree)

# def crawl_ingredient(pn):
#     # 화장품성분  
#     for n in range(5):
#         info = driver.find_elements_by_css_selector('section.section-list-item.ingredient.moreinfo div button')
#         sleep(1)
#         if len(info)>0:
#             info[0].click()
#             break
            
#     if len(info) == 0:
#         return []
            
#     measures = driver.find_element_by_css_selector('section.side-info.ingredient div.list-ingredient')
#     measures = measures.find_elements_by_class_name('list')

#     ings =[]
#     for m in measures:
#         soup = BeautifulSoup(m.get_attribute('innerHTML'), 'lxml')
#         icon = soup.select_one('div.list-item > span')
#         icontxt = icon.text
#         icon = icon['class'][2]

#         names = soup.select('div.list-item div.txt p')
#         narr = names[0].text.split('\n')
#         n1 = narr[0].strip()
#         n2 = narr[1].strip()
#         n3 = names[1].text.strip() 
#         #print(icontxt, icon)
#         ing = ingredient(pn, n1, n2, n3, icontxt, icon)
#         ings.append(ing)
#     return ings

# def crawl_reviews(pn):
#     driver.get('https://www.glowpick.com/product/{}'.format(pn))    
#     sleep(0.5)
    
#     uf = 'https://www.glowpick.com/product/{}?rating={}&order={}'
    
#     # 리뷰 개수 파악
#     count = '#gp-product-detail > div > ul.section-list-wrap.side-bottom > li.section-list-score.side-right > section > div > div.side-right > ul > li > div > p'
#     cnts = [int(cnt.text) for cnt in driver.find_elements_by_css_selector(count)]
    
#     # 리뷰 개수 체크용
#     count = '#gp-product-detail > div > ul.section-list-wrap.side-bottom > li.section-list-score.side-right > section > div > div.side-right > ul > li:nth-child({}) > div > p'
#     creviews = [] 
    
#     # 상품설명
#     exp = driver.find_element_by_css_selector('#gp-product-detail > div > ul.section-list-wrap.side-bottom > li.section-list-pdt.side-left > section > div > ul > li.desc > p').text

#     # 상품 키워드
#     tag = driver.find_element_by_css_selector('#gp-product-detail > div > ul.section-list-wrap.side-bottom > li.section-list-pdt.side-left > section > div > ul > li.desc > ul').text
    
#     errors = []
    
#     pdic = {'gpa-best': 5, 'gpa-good': 4, 'gpa-soso': 3, 'gpa-bad': 2, 'gpa-worst': 1}
#     #print(cnts)
#     for cnt, fn1, cidx in zip(cnts, range(5, 0, -1), range(1, 6)): 
#         if cnt == 0:
#             continue
                
#         loop = min(math.ceil(cnt/20), 5)
        
#         noloop1 = noloop2 = False
#         if cnt <= 100: noloop1 = True
#         if cnt <= 200: noloop2 = True
            
#         for idx, fn2 in enumerate(['create_date_asc','create_date_desc','like_desc','like_asc']):
#             url = uf.format(pn, fn1, fn2)
#             driver.get(url)
#             sleep(0.5)
            
#             # 앱으로보기 링크 있을경우 끄기
#             banners = driver.find_elements_by_xpath('//*[@id="top-downloadbanner"]/button')
#             if len(banners)>0:
#                 banners[0].click()
                
#             st = time()
#             # 리뷰 리스트가 없으면 찾을때까지 대기
#             while(True):
#                 rlist = driver.find_elements_by_class_name('review-list-wrap')
#                 if len(rlist)>0:
#                     if (rlist[0].text != ''):
#                         body = driver.find_element_by_css_selector('body')
#                         body.click()
#                         break
#                 else:
#                     driver.get(url)
#                     sleep(0.5)
                
#                 if time() - st > 30:
#                     errors.append([pn, fn1, fn2])
#                     break
                    
             
#             # 리뷰 개수가 일치해야 크롤링 시작
#             st = time()
#             while(True):
#                 ctxt = driver.find_element_by_css_selector(count.format(cidx)).text
#                 ctxt = int(ctxt)
#                 if cnt == ctxt:
#                     break
#                 else: 
#                     driver.get(url)
#                     sleep(0.5)
                
#                 if time() - st > 30:
#                     errors.append([pn, fn1, fn2])
#                     break
                    
            
#             # 최대 100개, 스크롤 해야 로드하기 때문에 end키 조작으로 리뷰 불러오기
#             st = time()
#             while(True):
#                 try:
#                     body.send_keys(Keys.END) 
#                     reviews = driver.find_elements_by_css_selector('ul.review-list-wrap li div.list-item') 

#                     if time() - st > 3:
#                         body.send_keys(Keys.PAGE_UP)
#                         sleep(0.1) 
#                         body.send_keys(Keys.END)
#                         st = time()

#                     if len(reviews) > (loop-1) * 20:
#                         break
                
#                 except Exception as e:
#                     body = driver.find_element_by_css_selector('body')
#                     body.click()
                 
                
#             #print(len(reviews))
#             # 리뷰 크롤링
#             # beautifulsoup이 좀더 빠르다
#             for r in reviews:
#                 soup = BeautifulSoup(r.get_attribute('innerHTML'), 'lxml')
#                 user = soup.select_one('.user .user-info')
#                 name = user.select_one('.name').text
#                 agetype = user.select_one('.info .txt')
#                 arr = agetype.text.split('·')
#                 age = int(arr[0].replace('세',''))
#                 ftype = arr[1] 
#                 sex = agetype.select_one('span')['class']
#                 point = user.select_one('.info .label span')['class'][1]
#                 point = pdic[point]

#                 comment = soup.select_one('.review').text

#                 cr = review(pn, name, age, ftype, sex, point, comment)
#                 creviews.append(cr)
            
#             if noloop1: break 
#             if noloop2 & (idx >= 1):break

#     #rank = driver.find_element_by_css_selector('#gp-product-detail > div > ul.section-list-wrap.side-top > li.section-list-img.side-left > div > section > ul > li > span').text
#     #print(rank) 
    
#     ing = crawl_ingredient(pn)
    
#     return exp, tag, creviews, ing, errors